<h1>ASSIGNMENT1</h1><h2>Download modules</h2>

In [0]:
!pip install SimpleITK
!pip install PyDrive
!pip install nilearn
!pip install git+https://www.github.com/farizrahman4u/keras-contrib.git

<h2>Import modules</h2>

In [0]:
import os
import requests
import shutil
import subprocess
import zipfile
from functools import partial

import keras
from keras.layers import Input, LeakyReLU, Add, SpatialDropout3D,Conv3D, MaxPooling3D, UpSampling3D, Activation, BatchNormalization, PReLU, Deconvolution3D
from keras.engine import Model
from keras.optimizers import Adam
from keras.models import Sequential

import tensorflow as tf
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt

from nilearn.image.image import check_niimg
from nilearn.image.image import _crop_img_to as crop_img_to

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials

from keras_contrib.layers.normalization import instancenormalization

<h2>Tunneling to TENSORBOARD</h2>

In [0]:
__all__ = [
  'install_ngrok', 
  'launch_tensorboard',
]

def __shell__(cmd, split=True):
  # get_ipython().system_raw(cmd)
  result = get_ipython().getoutput(cmd, split=split)
  if result and not split:
    result = result.strip('\n')
  return result  


# tested OK
def install_ngrok(bin_dir="/tmp"):
  """ download and install ngrok on local vm instance
  Args:
    bin_dir: full path for the target directory for the `ngrok` binary
  """
  TARGET_DIR = bin_dir
  CWD = os.getcwd()
  is_grok_avail = os.path.isfile(os.path.join(TARGET_DIR,'ngrok'))
  if is_grok_avail:
    print("ngrok installed")
  else:
    import platform
    plat = platform.platform() # 'Linux-4.4.64+-x86_64-with-Ubuntu-17.10-artful'
    if 'x86_64' in plat:
      
      os.chdir('/tmp')
      print("calling wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip ..." )
      get_ipython().system_raw( "wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip" )
      print("calling unzip ngrok-stable-linux-amd64.zip ...")
      get_ipython().system_raw( "unzip ngrok-stable-linux-amd64.zip" )
      os.rename("ngrok", "{}/ngrok".format(TARGET_DIR))
      os.remove("ngrok-stable-linux-amd64.zip")
      is_grok_avail = os.path.isfile(os.path.join(TARGET_DIR,'ngrok'))
      os.chdir(TARGET_DIR)
      if is_grok_avail:
        print("ngrok installed. path={}".format(os.path.join(TARGET_DIR,'ngrok')))
      else:
        # ValueError: ERROR: ngrok not found, path=
        raise ValueError( "ERROR: ngrok not found, path=".format(TARGET_DIR) )
    else:
      raise NotImplementedError( "ERROR, ngrok install not configured for this platform, platform={}".format(plat))
    os.chdir(CWD)
    return
    
# tested OK
def launch_tensorboard(bin_dir="/tmp", log_dir="/tmp", retval=False):
  """returns a public tensorboard url based on the ngrok package
  checks if `ngrok` is available, and installs, if necessary, to `bin_dir`
  launches tensorboard, if necessary
  see: https://stackoverflow.com/questions/47818822/can-i-use-tensorboard-with-google-colab
  Args:
    bin_dir: full path for the target directory for the `ngrok` binary
    log_dir: full path for the tensorflow `log_dir`
  Return:
    public url for tensorboard if retval==True
      NOTE: the method will print a link to stdout (cell output) for the tensorflow URL. 
      But the link printed from the return value has an extra "%27" in the URL which causes an error
  """
  install_ngrok(bin_dir)
    
  if not tf.gfile.Exists(log_dir):  tf.gfile.MakeDirs(log_dir)
  
  # check status of tensorboard and ngrok
  ps = __shell__("ps -ax")
  is_tensorboard_running = len([f for f in ps if "tensorboard" in f ]) > 0
  is_ngrok_running = len([f for f in ps if "ngrok" in f ]) > 0
  print("status: tensorboard={}, ngrok={}".format(is_tensorboard_running, is_ngrok_running))

  if not is_tensorboard_running:
    get_ipython().system_raw(
        'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
        .format(log_dir)
    )
    is_tensorboard_running = True
    
  if not is_ngrok_running:  
    #    grok should be installed in /tmp/ngrok
    get_ipython().system_raw('{}/ngrok http 6006 &'.format(bin_dir))
    is_ngrok_running = True

  # get tensorboard url
  # BUG: getting connection refused for HTTPConnectionPool(host='localhost', port=4040)
  #     on first run, retry works
  import time
  time.sleep(3)
  retval = requests.get('http://localhost:4040/api/tunnels')
  tensorboard_url = retval.json()['tunnels'][0]['public_url'].strip()
  print("tensorboard url=", tensorboard_url)
  if retval:
    return tensorboard_url

In [0]:
# set paths
ROOT = %pwd
LOG_DIR = os.path.join(ROOT, 'log1')

# will install `ngrok`, if necessary
# will create `log_dir` if path does not exist
launch_tensorboard( bin_dir=ROOT, log_dir=LOG_DIR )

<h2> Unzip in google drive</h2>(NOT IMPORTANT)

In [0]:
# drive.mount("/content/drive")
# with zipfile.ZipFile("/content/drive/My Drive/BRATS2015_Training.zip","r") as zip_ref:
#     zip_ref.extractall("/content/drive/My Drive/")

<h2>Local Unzip</h2>

In [0]:
## connect to drive and Locally download the file

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

download = drive.CreateFile({'id': '1AFiE99KilM3w9-f3BzhFSN_3_ztF5KTK'})
download.GetContentFile('BRATS2015.tar')

In [0]:
## Locally extract the zip

with zipfile.ZipFile("./BRATS2015.tar","r") as zip_ref:
    zip_ref.extractall("./BRATS2015")

<h2>All_Image_Sets --> Shuffle --> Test-Train_Split</h2>

In [0]:
## Handling directories
HGG_dir_list = next(os.walk('./BRATS2015/BRATS2015_Training/HGG/'))[1]
for i in range(len(HGG_dir_list)):
  HGG_dir_list[i]='./BRATS2015/BRATS2015_Training/HGG/'+HGG_dir_list[i]

LGG_dir_list = next(os.walk('./BRATS2015/BRATS2015_Training/LGG/'))[1]
for i in range(len(LGG_dir_list)):
  LGG_dir_list[i]='./BRATS2015/BRATS2015_Training/LGG/'+LGG_dir_list[i]


completelist = HGG_dir_list + LGG_dir_list

## shuffles in place
np.random.shuffle(completelist) 


## Train-Test dictionary
train_percentage=0.8

l0=0
l1=int(len(completelist)*train_percentage)+1
l2=len(completelist)

print(l1)
print(l2-l1)

partition={'train':np.empty((l1,5),dtype=object),'test':np.empty((l2-l1,5),dtype=object)}

for i in range(l1):
  a = next(os.walk(completelist[i]))[1]
  a.sort()
  for j in range(5):
    partition['train'][i][j]=completelist[i]+ '/'+a[j]+'/'+a[j]+'.mha'
    
for i in range(l1,l2):
  a = next(os.walk(completelist[i]))[1]
  a.sort()
  for j in range(5):
    partition['test'][i-l1][j]=completelist[i]+ '/'+a[j]+'/'+a[j]+'.mha'

<h2>mha --> numpy_3D_matrix</h2> 

In [0]:
def numpy_3D_matrix(ID):
  return(sitk.GetArrayFromImage(sitk.ReadImage(ID)))

In [0]:
## Example

ct_scan = numpy_3D_matrix(partition['test'][4][4])
np.shape(ct_scan)
plt.imshow(ct_scan[50,:,:])

<h1>MODEL</h1><h2>Pre-processing + U-Net + more filters + data augmentation + dice-loss + segmented image classification</h2>Refernces: https://arxiv.org/pdf/1802.10508v1.pdf , https://arxiv.org/pdf/1701.03056.pdf


<h2>Loss</h2>

In [0]:
def dice_coefficient(y_true, y_pred, smooth=1.):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coefficient_loss(y_true, y_pred):
    return -dice_coefficient(y_true, y_pred)


def weighted_dice_coefficient(y_true, y_pred, axis=(-3, -2, -1), smooth=0.00001):
    return K.mean(2. * (K.sum(y_true * y_pred,
                              axis=axis) + smooth/2)/(K.sum(y_true,
                                                            axis=axis) + K.sum(y_pred,
                                                                               axis=axis) + smooth))


def weighted_dice_coefficient_loss(y_true, y_pred):
    return -weighted_dice_coefficient(y_true, y_pred)


def label_wise_dice_coefficient(y_true, y_pred, label_index):
    return dice_coefficient(y_true[:, label_index], y_pred[:, label_index])


def get_label_dice_coefficient_function(label_index):
    f = partial(label_wise_dice_coefficient, label_index=label_index)
    f.__setattr__('__name__', 'label_{0}_dice_coef'.format(label_index))
    return f


dice_coef = dice_coefficient
dice_coef_loss = dice_coefficient_loss

<h2>Parts to create model</h2>

In [0]:
def create_convolution_block(input_layer, 
                             n_filters, 
                             batch_normalization=False, 
                             kernel=(3, 3, 3), 
                             activation=None,
                             padding='same',
                             strides=(1, 1, 1), 
                             instance_normalization=False):
    
    
    layer = Conv3D(n_filters, kernel, padding=padding, strides=strides)(input_layer)
    
    if batch_normalization:
        layer = BatchNormalization(axis=1)(layer)
    
    elif instance_normalization:
        layer = InstanceNormalization(axis=1)(layer)
    
    if activation is None:
        return Activation('relu')(layer)
    
    else:
        return activation()(layer)

In [0]:
def compute_level_output_shape(n_filters, depth, pool_size, image_shape):
    output_image_shape = np.asarray(np.divide(image_shape, np.power(pool_size, depth)), dtype=np.int32).tolist()
    return tuple([None, n_filters] + output_image_shape)

In [0]:
def get_up_convolution(n_filters, pool_size, kernel_size=(2, 2, 2), strides=(2, 2, 2),
                       deconvolution=False):
    if deconvolution:
        return Deconvolution3D(filters=n_filters, kernel_size=kernel_size,
                               strides=strides)
    else:
        return UpSampling3D(size=pool_size)

In [0]:
def create_localization_module(input_layer, n_filters):
    convolution1 = create_convolution_block(input_layer, n_filters)
    convolution2 = create_convolution_block(convolution1, n_filters, kernel=(1, 1, 1))
    return convolution2

In [0]:
def create_up_sampling_module(input_layer, n_filters, size=(2, 2, 2)):
    up_sample = UpSampling3D(size=size)(input_layer)
    convolution = create_convolution_block(up_sample, n_filters)
    return convolution

In [0]:
def create_context_module(input_layer, n_level_filters, dropout_rate=0.3, data_format="channels_first"):
    convolution1 = create_convolution_block(input_layer=input_layer, n_filters=n_level_filters)
    dropout = SpatialDropout3D(rate=dropout_rate, data_format=data_format)(convolution1)
    convolution2 = create_convolution_block(input_layer=dropout, n_filters=n_level_filters)
    return convolution2

In [0]:
create_convolution_block = partial(create_convolution_block, activation=LeakyReLU, instance_normalization=True)

<h2>Model</h2>

![alt text](https://ai2-s2-public.s3.amazonaws.com/figures/2017-08-08/f5702e1c095b99de32251967dab69ddab341736b/6-Figure4-1.png)

In [0]:
def MODEL(input_shape=(4, 1, 128, 128),
                      n_base_filters=16,
                      depth=5, 
                      dropout_rate=0.3,
                      n_segmentation_levels=3,
                      n_labels=4, 
                      optimizer=Adam, 
                      initial_learning_rate=5e-4,
                      loss_function=weighted_dice_coefficient_loss, 
                      activation_name="sigmoid"):    
    
    inputs = Input(input_shape)

    current_layer = inputs
    level_output_layers = list()
    level_filters = list()
    for level_number in range(depth):
        n_level_filters = (2**level_number) * n_base_filters
        level_filters.append(n_level_filters)

        if current_layer is inputs:
            in_conv = create_convolution_block(current_layer, n_level_filters)
        else:
            in_conv = create_convolution_block(current_layer, n_level_filters, strides=(2, 2, 2))

        context_output_layer = create_context_module(in_conv, n_level_filters, dropout_rate=dropout_rate)

        summation_layer = Add()([in_conv, context_output_layer])
        level_output_layers.append(summation_layer)
        current_layer = summation_layer

    segmentation_layers = list()
    for level_number in range(depth - 2, -1, -1):
        up_sampling = create_up_sampling_module(current_layer, level_filters[level_number])
        concatenation_layer = concatenate([level_output_layers[level_number], up_sampling], axis=1)
        localization_output = create_localization_module(concatenation_layer, level_filters[level_number])
        current_layer = localization_output
        if level_number < n_segmentation_levels:
            segmentation_layers.insert(0, create_convolution_block(current_layer, n_filters=n_labels, kernel=(1, 1, 1)))

    output_layer = None
    for level_number in reversed(range(n_segmentation_levels)):
        segmentation_layer = segmentation_layers[level_number]
        if output_layer is None:
            output_layer = segmentation_layer
        else:
            output_layer = Add()([output_layer, segmentation_layer])

        if level_number > 0:
            output_layer = UpSampling3D(size=(2, 2, 2))(output_layer)

    activation_block = Activation(activation_name)(output_layer)
    survival_block = Activation("linear")(summation_layer)

    model = Model(inputs=inputs, outputs=[activation_block,survival_block])
    model.compile(optimizer=optimizer(lr=initial_learning_rate), loss=loss_function)
    return model

In [0]:
model = MODEL(input_shape=(4, 1, 192, 160), n_base_filters=6, depth=5, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=3, optimizer=Adam, initial_learning_rate=5e-4,
                      loss_function=weighted_dice_coefficient_loss, activation_name="sigmoid")
model.summary()

<h2>Training MODEL</h2>

In [0]:
params = {'dim': (160,192,160),
          'batch_size': 2,
          'n_classes': 3,
          'n_channels': 4,
          'shuffle': True}

training_generator = 
validation_generator = 

cb_1=keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')
cb_2=keras.callbacks.ModelCheckpoint(filepath=weights.{epoch:02d}-{val_loss:.2f}.hdf5, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

results = model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                   epochs=2, 
                   nb_worker=4,
                   callbacks=[cb_1,cb_2])
print("Test-Accuracy:", np.mean(results.history["val_acc"]))

<h2> Testing Model </h2>

In [0]:
params = {'dim': (160,192,160),
          'batch_size': 2,
          'n_classes': 3,
          'n_channels': 4,
          'shuffle': False}

validation_generator = DataGenerator(partition['holdout'], labels, **params)

prediction = model.predict_generator(generator=validation_generator)

prediction